In [ ]:
!pip install -q "transformers>=4.46.0" "datasets>=3.0.0" sacrebleu sentencepiece accelerate


In [ ]:

# FINE-TUNE MARIANMT TRÊN DỮ LIỆU MEDICAL VLSP (EN↔VI)
import os
from pathlib import Path
import random

import numpy as np
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import sacrebleu

os.environ["WANDB_DISABLED"] = "true"

print("Transformers version:", __import__("transformers").__version__)
print("CUDA available:", torch.cuda.is_available())

# Set seed cho reproducible
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)


Transformers version: 4.57.3
CUDA available: True


In [ ]:

# 1. CẤU HÌNH ĐƯỜNG DẪN & HƯỚNG DỊCH

DATA_DIR = Path("/content")

# Hướng dịch: "en-vi" hoặc "vi-en"
DIRECTION = "en-vi"   # <<< ĐỔI Ở ĐÂY NẾU MUỐN VI→EN

if DIRECTION == "en-vi":
    TRAIN_SRC_FILE = DATA_DIR / "train.en.txt"
    TRAIN_TGT_FILE = DATA_DIR / "train.vi.txt"
    TEST_SRC_FILE  = DATA_DIR / "public_test.en.txt"
    TEST_TGT_FILE  = DATA_DIR / "public_test.vi.txt"
    MODEL_NAME = "Helsinki-NLP/opus-mt-en-vi"
elif DIRECTION == "vi-en":
    TRAIN_SRC_FILE = DATA_DIR / "train.vi.txt"
    TRAIN_TGT_FILE = DATA_DIR / "train.en.txt"
    TEST_SRC_FILE  = DATA_DIR / "public_test.vi.txt"
    TEST_TGT_FILE  = DATA_DIR / "public_test.en.txt"
    MODEL_NAME = "Helsinki-NLP/opus-mt-vi-en"
else:
    raise ValueError("DIRECTION phải là 'en-vi' hoặc 'vi-en'")

print("Direction:", DIRECTION)
print("Model:", MODEL_NAME)
print("Train src:", TRAIN_SRC_FILE)
print("Train tgt:", TRAIN_TGT_FILE)
print("Test  src:", TEST_SRC_FILE)
print("Test  tgt:", TEST_TGT_FILE)


Direction: en-vi
Model: Helsinki-NLP/opus-mt-en-vi
Train src: /content/train.en.txt
Train tgt: /content/train.vi.txt
Test  src: /content/public_test.en.txt
Test  tgt: /content/public_test.vi.txt


In [ ]:

# 2. ĐỌC CORPUS SONG NGỮ (ĐÃ SỬA)

def load_parallel_corpus(src_path: Path, tgt_path: Path, max_samples: int = None):
    """Đọc 2 file song ngữ dòng-đối-dòng.
    - Nếu 2 file dài khác nhau -> cắt theo min_len để giữ alignment.
    - max_samples: giới hạn số cặp để train cho nhẹ (None nếu dùng hết).
    """
    with open(src_path, encoding="utf-8") as f_src:
        src_lines = [l.strip() for l in f_src.readlines()]

    with open(tgt_path, encoding="utf-8") as f_tgt:
        tgt_lines = [l.strip() for l in f_tgt.readlines()]

    if max_samples is not None:
        src_lines = src_lines[:max_samples]
        tgt_lines = tgt_lines[:max_samples]

    if len(src_lines) != len(tgt_lines):
        min_len = min(len(src_lines), len(tgt_lines))
        print(
            f"⚠️ CẢNH BÁO: source có {len(src_lines)} dòng, "
            f"target có {len(tgt_lines)} dòng. "
            f"Chỉ dùng {min_len} cặp đầu tiên."
        )
        src_lines = src_lines[:min_len]
        tgt_lines = tgt_lines[:min_len]

    pairs = []
    for s, t in zip(src_lines, tgt_lines):
        if s and t:
            pairs.append((s, t))

    src, tgt = zip(*pairs)
    return list(src), list(tgt)


# ==== GỌI HÀM ĐỂ ĐỌC DỮ LIỆU ====

# Nếu muốn giới hạn số lượng train cho nhẹ máy, đặt số ở đây (ví dụ 200000),
# còn muốn dùng hết thì để None
MAX_TRAIN_SAMPLES = None   # hoặc 200000 / 100000 tuỳ GPU

train_src, train_tgt = load_parallel_corpus(
    TRAIN_SRC_FILE, TRAIN_TGT_FILE, max_samples=MAX_TRAIN_SAMPLES
)
test_src,  test_tgt  = load_parallel_corpus(
    TEST_SRC_FILE,  TEST_TGT_FILE,  max_samples=None
)

print(f"Loaded {len(train_src)} training pairs")
print(f"Loaded {len(test_src)} test pairs")

print("\nVí dụ train:")
for i in range(3):
    print("SRC:", train_src[i][:120])
    print("TGT:", train_tgt[i][:120])
    print("-" * 60)


Loaded 500000 training pairs
Loaded 3000 test pairs

Ví dụ train:
SRC: To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology de
TGT: Nghiên cứu đặc điểm lâm sàng, cận lâm sàng bệnh nhân viêm tai ứ dịch trên viêm V.A tại Khoa Tai mũi họng - Bệnh viện Tru
------------------------------------------------------------
SRC: Evaluate clinical, subclinical symptoms of patients with otittis media effusion and V a at otorhinolaryngology departmen
TGT: Đánh giá đặc điểm lâm sàng, cận lâm sàng bệnh nhân viêm tai ứ dịch trên viêm V.a tại Khoa Tai mũi họng - Bệnh viện Trung
------------------------------------------------------------
SRC: There was a relation between vasodilatation and vaginal dysfunction.
TGT: Có sự liên quan giữa độ quá phát V.a với mức độ rối loạn chức năng vòi nhĩ.
------------------------------------------------------------


In [ ]:

# 3. TẠO DATASETDICT
raw_train = Dataset.from_dict({"src": train_src, "tgt": train_tgt})
raw_test  = Dataset.from_dict({"src": test_src,  "tgt": test_tgt})

# Lấy 5% train làm validation
train_valid = raw_train.train_test_split(test_size=0.05, seed=42)

dataset = DatasetDict(
    {
        "train":      train_valid["train"],
        "validation": train_valid["test"],
        "test":       raw_test,
    }
)

print(dataset)
print(dataset["train"][0])


DatasetDict({
    train: Dataset({
        features: ['src', 'tgt'],
        num_rows: 475000
    })
    validation: Dataset({
        features: ['src', 'tgt'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['src', 'tgt'],
        num_rows: 3000
    })
})
{'src': 'Conclusions: Communication skills was taught to the whole of second-year students in Hanoi Medical University at the year 2013 - 2014 with 2 credits.', 'tgt': 'Kết luận: Môn học KNGT đã được giảng cho toàn bộ khối sinh viên Y2 tại trường Đại học Y Hà Nội từ năm học 2013-2014 với cấu trúc 1/1.'}


In [ ]:
# ==========================
# 4. TOKENIZER & PREPROCESS
# ==========================

MAX_SOURCE_LENGTH = 128   # có thể tăng 256 nếu câu dài và GPU đủ khỏe
MAX_TARGET_LENGTH = 128

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(batch):
    # Encode source
    model_inputs = tokenizer(
        batch["src"],
        max_length=MAX_SOURCE_LENGTH,
        truncation=True,
    )

    # Encode target
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["tgt"],
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["src", "tgt"],
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=MODEL_NAME,
)

tokenized_datasets


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

...

Map:   0%|          | 0/475000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 475000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
})

In [ ]:

# 5. MODEL & HÀM TÍNH BLEU

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

def postprocess_text(preds, labels):
    """Làm sạch chuỗi trước khi tính BLEU."""
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]   # list[str], KHÔNG bọc thêm []
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Seq2SeqTrainer đôi khi trả về tuple
    if isinstance(preds, tuple):
        preds = preds[0]

    # Decode predictions
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Chuyển -100 -> pad_token_id để decode được labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # sacrebleu expects: hyps = list[str], refs = list[list[str]]
    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])

    # Độ dài trung bình
    prediction_lens = [
        np.count_nonzero(p != tokenizer.pad_token_id) for p in preds
    ]

    result = {
        "bleu": round(bleu.score, 4),
        "gen_len": float(np.mean(prediction_lens)),
    }
    return result


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:

# 5. MODEL & HÀM TÍNH BLEU


model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

def postprocess_text(preds, labels):
    """Làm sạch chuỗi trước khi tính BLEU."""
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]   # list[str], KHÔNG bọc thêm []
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Seq2SeqTrainer đôi khi trả về tuple
    if isinstance(preds, tuple):
        preds = preds[0]

    # Decode predictions
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Chuyển -100 -> pad_token_id để decode được labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # sacrebleu expects: hyps = list[str], refs = list[list[str]]
    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])

    # Độ dài trung bình
    prediction_lens = [
        np.count_nonzero(p != tokenizer.pad_token_id) for p in preds
    ]

    result = {
        "bleu": round(bleu.score, 4),
        "gen_len": float(np.mean(prediction_lens)),
    }
    return result


model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

In [ ]:

# 6. TRAINING ARGUMENTS & TRAINER

batch_size = 8
num_epochs = 2

output_dir = f"vlsp_marian_{DIRECTION}"

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,

    # NEW API: eval_strategy (evaluation_strategy đã bị bỏ)
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,

    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=num_epochs,

    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LENGTH,
    generation_num_beams=4,

    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,

    fp16=torch.cuda.is_available(),
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-2320806084.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# ==========================
# 7. HUẤN LUYỆN
# ==========================

train_result = trainer.train()
trainer.save_model(output_dir)

metrics = train_result.metrics
metrics["train_samples"] = len(tokenized_datasets["train"])
print("Train metrics:", metrics)

# Lưu log để sau này vẽ đồ thị Loss/BLEU cho báo cáo
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.321500,1.219266,46.969800,35.794600


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[53684]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.321500,1.219266,46.969800,35.794600
2,1.174800,1.110456,49.212100,36.061720


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


Train metrics: {'train_runtime': 11586.7682, 'train_samples_per_second': 81.99, 'train_steps_per_second': 10.249, 'total_flos': 1.812094088852275e+16, 'train_loss': 1.3704857842053866, 'epoch': 2.0, 'train_samples': 475000}
***** train metrics *****
  epoch                    =        2.0
  total_flos               = 16876441GF
  train_loss               =     1.3705
  train_runtime            = 3:13:06.76
  train_samples            =     475000
  train_samples_per_second =      81.99
  train_steps_per_second   =     10.249


In [ ]:
# ==========================
# 8. ĐÁNH GIÁ TRÊN TEST (BLEU)
# ==========================

metrics = trainer.evaluate(tokenized_datasets["test"])
metrics["test_samples"] = len(tokenized_datasets["test"])

print("Test metrics:", metrics)

trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)


Test metrics: {'eval_loss': 1.161577582359314, 'eval_bleu': 47.4958, 'eval_gen_len': 36.03366666666667, 'eval_runtime': 325.2827, 'eval_samples_per_second': 9.223, 'eval_steps_per_second': 1.153, 'epoch': 2.0, 'test_samples': 3000}
***** test metrics *****
  epoch                   =        2.0
  eval_bleu               =    47.4958
  eval_gen_len            =    36.0337
  eval_loss               =     1.1616
  eval_runtime            = 0:05:25.28
  eval_samples_per_second =      9.223
  eval_steps_per_second   =      1.153
  test_samples            =       3000


In [ ]:
# ==========================
# 9. DỊCH FILE TEST & LƯU KẾT QUẢ + VÍ DỤ ERROR ANALYSIS
# ==========================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def translate_sentences(sentences, batch_size=16, num_beams=4):
    all_trans = []
    model.eval()
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i : i + batch_size]
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_SOURCE_LENGTH,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_length=MAX_TARGET_LENGTH,
                num_beams=num_beams,
            )

        batch_trans = tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True
        )
        all_trans.extend(batch_trans)

    return all_trans


preds = translate_sentences(test_src, batch_size=16, num_beams=4)

# Lưu ra file dự đoán
pred_file = os.path.join(output_dir, f"pred_{DIRECTION}.txt")
with open(pred_file, "w", encoding="utf-8") as f:
    for line in preds:
        f.write(line + "\n")

print("Saved predictions to:", pred_file)

# Tính BLEU lại bằng sacrebleu (double-check)
detok_preds, detok_refs = postprocess_text(preds, test_tgt)
bleu_test = sacrebleu.corpus_bleu(detok_preds, [detok_refs])
print(f"BLEU trên tập test (tính trực tiếp): {bleu_test.score:.4f}")

# In vài ví dụ để phân tích lỗi trong báo cáo
for i in range(5):
    print("=" * 80)
    print("SRC :", test_src[i])
    print("PRED:", preds[i])
    print("REF :", test_tgt[i])


Saved predictions to: vlsp_marian_en-vi/pred_en-vi.txt
BLEU trên tập test (tính trực tiếp): 47.1984
SRC : Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao
PRED: Kiến thức, thực hành về sử dụng dịch vụ y tế công cộng của người quản lý thẻ bảo hiểm y tế và các yếu tố ảnh hưởng tại Việt Nam, Lào
REF : Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017
SRC : Describe knowledge, practices in public health service utilization among health insurance card's holders and influencing factors in Vientiane, Lao PDR, 2017.
PRED: Mô tả kiến thức, thực hành về sử dụng dịch vụ y tế công cộng của người giữ thẻ bảo hiểm y tế và các yếu tố ảnh hưởng ở Việt Nam, Lào PDR, 2017.
REF : Mô tả thực trạng kiến thức, thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám ch

In [ ]:
!ls -R vlsp_marian_en-vi



vlsp_marian_en-vi:
all_results.json	pred_en-vi.txt		 trainer_state.json
checkpoint-118750	source.spm		 training_args.bin
checkpoint-59375	special_tokens_map.json  train_results.json
config.json		target.spm		 vocab.json
generation_config.json	test_results.json
model.safetensors	tokenizer_config.json

vlsp_marian_en-vi/checkpoint-118750:
config.json		scaler.pt		 tokenizer_config.json
generation_config.json	scheduler.pt		 trainer_state.json
model.safetensors	source.spm		 training_args.bin
optimizer.pt		special_tokens_map.json  vocab.json
rng_state.pth		target.spm

vlsp_marian_en-vi/checkpoint-59375:
config.json		scaler.pt		 tokenizer_config.json
generation_config.json	scheduler.pt		 trainer_state.json
model.safetensors	source.spm		 training_args.bin
optimizer.pt		special_tokens_map.json  vocab.json
rng_state.pth		target.spm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục chứa model trên Drive (tùy bạn đặt tên)
!mkdir -p "/content/drive/MyDrive/BTL_NLP_MT"

# Copy toàn bộ model sang Drive
!cp -r vlsp_marian_en-vi "/content/drive/MyDrive/BTL_NLP_MT/vlsp_marian_en-vi"


MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Nén cả thư mục model thành 1 file .zip
!zip -r vlsp_marian_en-vi.zip vlsp_marian_en-vi


  adding: vlsp_marian_en-vi/ (stored 0%)
  adding: vlsp_marian_en-vi/trainer_state.json (deflated 84%)
  adding: vlsp_marian_en-vi/pred_en-vi.txt (deflated 72%)
  adding: vlsp_marian_en-vi/checkpoint-118750/ (stored 0%)
  adding: vlsp_marian_en-vi/checkpoint-118750/trainer_state.json (deflated 84%)
  adding: vlsp_marian_en-vi/checkpoint-118750/rng_state.pth (deflated 26%)
  adding: vlsp_marian_en-vi/checkpoint-118750/special_tokens_map.json (deflated 35%)
  adding: vlsp_marian_en-vi/checkpoint-118750/vocab.json (deflated 70%)
  adding: vlsp_marian_en-vi/checkpoint-118750/target.spm (deflated 50%)
  adding: vlsp_marian_en-vi/checkpoint-118750/scheduler.pt (deflated 61%)
  adding: vlsp_marian_en-vi/checkpoint-118750/config.json (deflated 63%)
  adding: vlsp_marian_en-vi/checkpoint-118750/training_args.bin (deflated 53%)
  adding: vlsp_marian_en-vi/checkpoint-118750/model.safetensors (deflated 7%)
  adding: vlsp_marian_en-vi/checkpoint-118750/tokenizer_config.json (deflated 68%)
  adding:

In [ ]:
from google.colab import files
files.download("vlsp_marian_en-vi.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục chứa model trên Drive (tùy bạn đặt tên)
!mkdir -p "/content/drive/MyDrive/BTL_NLP_MT"

# Copy toàn bộ model sang Drive
!cp -r vlsp_marian_en-vi "/content/drive/MyDrive/BTL_NLP_MT/vlsp_marian_en-vi"


Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Nếu model nằm trong Drive:
model_dir = "/content/drive/MyDrive/BTL_NLP_MT/vlsp_marian_en-vi"

# Nếu bạn vừa giải nén zip trong /content:
# model_dir = "/content/vlsp_marian_en-vi"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir).to(device)


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
def translate_en_vi(sentence: str, num_beams: int = 4, max_length: int = 128) -> str:
    model.eval()
    inputs = tokenizer(
        [sentence],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length,
    ).to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            num_beams=num_beams,
            max_length=max_length,
        )

    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)


print(translate_en_vi("This is a medical sentence about diabetes."))


Đây là một câu hỏi y khoa về bệnh đái tháo đường.
